In [6]:
import pandas as pd
import ast

from tqdm.notebook import tqdm

In [41]:
edam = pd.read_csv('EDAM/EDAM.csv')
clinepidb = pd.read_csv('datasets/CLINEPIDB.csv')

In [42]:
data = pd.read_csv("outputs/updated_scoring_results_clinepidb2.csv")

In [45]:
import csv
import json
import ast

# Initialize an empty list to store topicCategory objects
topic_categories = []

# Read the CSV file
for _, row in data.iterrows():
    # Extract data from the row
    output = ast.literal_eval(row['Predictions'])

    defined_terms = []
    for term in output:
        # Create a DefinedTerm object
        defined_term = {
            "definedTerm": {
                "@type": "DefinedTerm",
                "type": "object",
                "properties": {
                    "name": term,
                    "url": edam[edam['Preferred Label'] == term]['Class ID'].values[0],
                    "curatedBy": {
                        "properties": {
                            "name": "GPT3.5"  # Replace "Curator Name" with the actual curator name
                        }
                    },
                    "inDefinedTermSet": "EDAM",
                }
            }
        }
        defined_terms.append(defined_term)

    # Create a topicCategory object
    topic_category = {
        "topicCategory": {
            "properties": {
                "name": row['Name'],
                # "description": row['Description'],
                "url": row['URL'],
                "curatedBy": {
                    "properties": {
                        "name": "GPT3.5",  # Replace "Curator Name" with the actual curator name
                    }
                },
                "categorizedTopics": defined_terms  # Add the DefinedTerm object to the topicCategory
            }
        }
    }

    # Append the topicCategory object to the list
    topic_categories.append(topic_category)

# Convert the list of topicCategory objects to JSON
json_output = json.dumps(topic_categories, indent=4)

# Print or save the JSON output
print(json_output)

[
    {
        "topicCategory": {
            "properties": {
                "name": "SHINE Cluster Randomized Trial",
                "url": "https://clinepidb.org/ce/app/record/dataset/DS_0086998c2f",
                "curatedBy": {
                    "properties": {
                        "name": "GPT3.5"
                    }
                },
                "categorizedTopics": [
                    {
                        "definedTerm": {
                            "@type": "DefinedTerm",
                            "type": "object",
                            "properties": {
                                "name": "Public health and epidemiology",
                                "url": "http://edamontology.org/topic_3305",
                                "curatedBy": {
                                    "properties": {
                                        "name": "GPT3.5"
                                    }
                                },
                      

In [46]:
import json

# Specify the file path
file_path = 'outputs/clinepidb.json'

# Write the JSON output to the file
with open(file_path, 'w') as file:
    json.dump(topic_categories, file, indent=4)


## Convert to JSON

[
{"VDJ_ID": ["prediction 1", "prediction 2"]},
{"VDJ_ID2": ["prediction 1", "prediction 2"]},
...
]

In [24]:
import os
import glob
import json

folder_path = 'outputs/processed_outputs/'  # Replace with the actual folder path
file_paths = glob.glob(folder_path + '/*_processed*')

In [25]:
file_paths

['outputs/processed_outputs/qiita_processed.csv',
 'outputs/processed_outputs/clinepidb_processed.csv',
 'outputs/processed_outputs/accessclinicaldata_processed.csv',
 'outputs/processed_outputs/dryad_processed.csv',
 'outputs/processed_outputs/immport_processed.csv',
 'outputs/processed_outputs/harvard_processed.csv',
 'outputs/processed_outputs/vdjserver_processed.csv',
 'outputs/processed_outputs/hubmap_processed.csv',
 'outputs/processed_outputs/datadiscoveryengine_processed.csv',
 'outputs/processed_outputs/lincs_processed.csv',
 'outputs/processed_outputs/mendeley_processed.csv',
 'outputs/processed_outputs/ncbi_geo_processed.csv',
 'outputs/processed_outputs/ncbi_sra_processed.csv',
 'outputs/processed_outputs/omicsdi_processed.csv',
 'outputs/processed_outputs/reframedb_processed.csv',
 'outputs/processed_outputs/veupathdb_processed.csv',
 'outputs/processed_outputs/vivli_processed.csv',
 'outputs/processed_outputs/zenodo_processed.csv']

In [26]:
for file_path in tqdm(file_paths):
    data = pd.read_csv(file_path, lineterminator='\n')

    file_base_name = os.path.basename(file_path)
    file_base_name = os.path.splitext(file_base_name)[0]

    data['Predictions'] = data['Predictions'].apply(ast.literal_eval).apply(list)
    output = []
    for idx, row in tqdm(data.iterrows(), total=data.shape[0]):
        output.append({row['_id']: row['Predictions']})
        
    # Specify the file path
    dump_path = f'outputs/jsons/{file_base_name}.json'

    # Write the JSON output to the file
    with open(dump_path, 'w') as file:
        json.dump(output, file, indent=4)

